In [1]:
from defusedxml import ElementTree as DET
import xml.etree.ElementTree as ET
import pandas as pd
import httpx
import uuid

In [53]:
# completed
r = httpx.get('https://legalassistance.law.af.mil/AMJAMS/PublicDocket/chargeList.xml')

tree = ET.ElementTree(DET.fromstring(r.text)).getroot()
data = []


for element in tree:
    code = element.find("specCode").text
    article = element.find("specArticle").text
    definition = element.find("specDefinition").text

    data.append({"code": code, "article": article, "definition": definition})

articles = pd.DataFrame(data)
articles

,code,article,definition
0,081-A-,81,"Conspiracy; war crime, lead to death"
1,082---,82,Solicitation of an offense
2,082-B1,82,Solicit desertion
3,082-B2,82,Solicit mutiny
4,082-B3,82,Solicit misbehavior before enemy
...,...,...,...
766,134-Z2,134,Other offenses under Article 134
767,134R6A,134,Child pornography; possession
768,134R6B,134,Child pornography; intent to distribute
769,134R6C,134,Child pornography; distribution


In [54]:
# completed

r = httpx.get('https://legalassistance.law.af.mil/AMJAMS/PublicDocket/baseList.xml')

tree = ET.ElementTree(DET.fromstring(r.text)).getroot()
data  = []

for element in tree:
    base = element.find("base").text
    base_name = element.find("baseName").text
    state_abv = element.find("stateAbbrev").text
    state_long = element.find("stateLongName").text
    phone = element.find("basePhone").text

    data.append(
        {
            "base": base,
            "base_name": base_name,
            "state_abv": state_abv,
            "state": state_long,
            "phone_number": phone,
        }
    )
bases = pd.DataFrame(data)
bases

,base,base_name,state_abv,state,phone_number
0,None,UNDEFINED,XX,Undefined Location,000-000-0000
1,ATU2,USAFCENT - 466 AEG,AE,Deployed Location,803-895-2875
2,ATUH,USAFCENT - 455 AEW,AE,Deployed Location,803-895-2875
3,FBKA,USAFCENT - 451 AEW,AE,Deployed Location,803-895-2875
4,FDMY,USAFCENT - 376 AEW,AE,Deployed Location,803-895-2875
...,...,...,...,...,...
106,TRC1,Vandenberg SFB,CA,California,805-606-5395
107,TRC2,Peterson-Schreiver Garrison,CO,Colorado,719-556-5815
108,TRC3,Peterson-Schreiver Garrison,CO,Colorado,719-556-5815
109,TRC4,Vandenberg SFB,CA,California,805-606-5395


In [55]:
r = httpx.get('https://legalassistance.law.af.mil/AMJAMS/PublicDocket/caseList.1.7.xml')

root = ET.fromstring(r.text)

element_data = []
sub_element_data = []


# loop through each element, and sub element and parse data
for tree in root:
    for element in tree:
        # Generate a uniue id for each element
        id: str = str(uuid.uuid1())

        element.attrib["fk_id"] = id

        case_info: dict = {element.tag: element.attrib}

        for sub_element in element:
            sub_element.attrib["case_id"] = id

            sub_element_data.append(sub_element.attrib)

        element_data.append(case_info)

case_charges = []
case_personnel = []

# Loop through the sub elements and parse data

for values in sub_element_data:
    if "code" in values:
        case_charge_data = {
            "case_id": values.get("case_id"),
            "code": values.get("code"),
            "prefix": values.get("prefix"),
        }

        case_charges.append(case_charge_data)

    else:
        case_personnel_data = {
            "case_id": values.get("case_id"),
            "order": values.get("order"),
            "title": values.get("title"),
            "name": values.get("name"),
        }

        case_personnel.append(case_personnel_data)

df_case_info = pd.json_normalize(element_data)
df_case_charges = pd.DataFrame(case_charges)
df_case_personnel = pd.DataFrame(case_personnel)


In [56]:
df_case_charges

,case_id,code,prefix
0,75cec36c-c156-11ef-9588-cafaa44913ac,112AC1,
1,75cec4ca-c156-11ef-9588-cafaa44913ac,087AA2,
2,75cec4ca-c156-11ef-9588-cafaa44913ac,091-C2,
3,75cec4ca-c156-11ef-9588-cafaa44913ac,092-B-,
4,75cec4ca-c156-11ef-9588-cafaa44913ac,112AC4,
...,...,...,...
4516,75cfdfb8-c156-11ef-9588-cafaa44913ac,112AC1,
4517,75cfdfb8-c156-11ef-9588-cafaa44913ac,112AD1,
4518,75cfdfb8-c156-11ef-9588-cafaa44913ac,118-A-,Conspiracy to commit -
4519,75cfdfb8-c156-11ef-9588-cafaa44913ac,126-A-,


In [89]:
# for the first 2 xml files, everything is a child of the root. there are three sub-elements in this xml file.

r = httpx.get('https://legalassistance.law.af.mil/AMJAMS/PublicDocket/caseList.1.7.xml')

element = ET.ElementTree(DET.fromstring(r.text)).getroot().find('trialResults')

In [117]:
for i in element:

    # print(i.attrib)

    # personell = [x.attrib for x in i.findall('trialPersonnel')]
    # chrg = [x.attrib for x in i.findall('charge')]
    # print(personell)


    for a in i:
        print(a)


<Element 'charge' at 0x15db19210>
<Element 'trialPersonnel' at 0x15db191c0>
<Element 'trialPersonnel' at 0x15db18e50>
<Element 'trialPersonnel' at 0x15db19120>
<Element 'trialPersonnel' at 0x15db19080>
<Element 'charge' at 0x15db18fe0>
<Element 'charge' at 0x15db18f90>
<Element 'trialPersonnel' at 0x15db18ef0>
<Element 'trialPersonnel' at 0x15db18ea0>
<Element 'trialPersonnel' at 0x15db18c20>
<Element 'trialPersonnel' at 0x15db18d60>
<Element 'charge' at 0x15db18c70>
<Element 'charge' at 0x15db189a0>
<Element 'trialPersonnel' at 0x15db18bd0>
<Element 'trialPersonnel' at 0x15db18b80>
<Element 'trialPersonnel' at 0x15db18b30>
<Element 'trialPersonnel' at 0x15db18a90>
<Element 'charge' at 0x15db18720>
<Element 'trialPersonnel' at 0x15db18950>
<Element 'trialPersonnel' at 0x15db18900>
<Element 'trialPersonnel' at 0x15db188b0>
<Element 'trialPersonnel' at 0x15db18810>
<Element 'trialPersonnel' at 0x15db18450>
<Element 'trialPersonnel' at 0x15db186d0>
<Element 'trialPersonnel' at 0x15db18630

In [101]:
for neighbor in element.iter('neighbor'):
    print(neighbor.attrib)